In [15]:
from jpnotebooks.other.wql import *
import collections
import types
import datetime
import itertools
import time
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
MIDNA = 'Mîdna'
SERVER = 'Turalyon'
OWNER = "nintendobratkat"
GUILD = "Synthesis"
ZONE = "Tomb of Sargeras"
MYTHIC = 5
api_key = "192b298b50ad53497d9f2b4ce7910af2"

try:
    zraw
except NameError:
    zraw = Api(api_key).zones._raw
    
def connect(api_key=api_key):
    global api
    api = Api(api_key)
    api.zones_ = Zones(api, zraw)
    return api

def pd(o):  # printdir
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o,a,"<Error>")
            if not isinstance(v, (types.FunctionType, types.MethodType)):
                print(a, "::", v)

                
# TODO: move these to wql.py once I'm confident they works properly


def _clstrfk(events, l):
    try:
        _clstrfk2(events, l)
    except ValueError:
        for c in l:
            for e in c:
                print(e.timestamp, actor(f, e))
            print()
    return l
    
def _clstrfk2(events, l):
    # debug sanity check
    seen = set()
    evd = dict.fromkeys(events, False)
    for c in l:
        for e in c:
            if evd[e] == True:
                raise ValueError("Duplicate event!")
            evd[e] = True
            
def cluster2(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    _clstrfk(events, l)
    return sorted(map(list,l), key=lambda c: c[0].timestamp)

def cluster(events, before=100, after=100):
    l = cluster2(events, before, after)
    l = [(c[0].timestamp, c) for c in l]
    return sorted(l, key=lambda t: t[0])
   
def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)
                
connect()
api.verbose = False

In [17]:
def deaths_near_bad(f, *bad, before=100, after=10000, cutoff=8):
    fs_deaths = 'type="death"'
    deaths = [e for e in f.events(fs_deaths, cutoff=cutoff) if e.targetID and f.actor(e.targetID).is_player()]
    fs_bads = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    bads = f.events(fs_bads, cutoff=cutoff)
    fails = set()
    for d in deaths:
        t = tuple(events_near(bads, d, before, after)), d
        fails.add(t)
    return fails

def deaths_near_bad2(f, *bad, before=100, after=10000, cutoff=8):
    fails = deaths_near_bad(f, *bad, before=before, after=after, cutoff=cutoff)
    fails2 = set()
    deaths = set()
    for ev, d in fails:
        fails2.update(ev)
        deaths.add(d)
    key = lambda e: e.timestamp
    return sorted(fails2, key=key), sorted(deaths, key=key)

def hit_by_bad(f, *bad, cutoff=8):
    fs = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    return f.events(fs, cutoff=cutoff)

In [18]:
cache = {}
def get(url):
    global cache
    if url in cache:
        return cache[url]
    rv = api.sess.request("GET", url).content.decode()
    cache[url] = rv
    return rv
def query(s):
    return pyquery.PyQuery(s)("#reports-table > tbody > tr")

In [19]:
def throttle(t):
    def _throttle(f):
        l = time.time()
        n = l + t
        def func(*args, **kw):
            c = time.time()
            s = n-c
            if s > 0:
                time.sleep(s)
            try:
                rv = f(*args, **kw)
            finally:
                l = c
                n = c + t
            return rv
        return func
    return _throttle

@throttle(1/3)
def hf(id_):
    fights = api.fights(id_)
    if fights.find(kill=True, difficulty=5, boss=2038).fights:
        return True

In [20]:
reports =api.reports(GUILD, SERVER, "US")
report_margus = reports.find(id="nC1YMGzb9kQwdDV6")
fights = report_margus.reports[0].fights()
f = fights.fights[0]

In [124]:
def event_damage(*abilities):
    tmpl = "ability.name=\"%s\""
    al = [tmpl%a for a in abilities]
    base = " OR ".join(al)
    return "(type=\"damage\" AND (%s))"%base
    
def crushing_fear():
    cf = 'ability.name="Crushing Fear"'
    tmpl = '(type="%s")'
    evt = "damage", "applydebuff", "applydebuffstack", "removedebuff"
    fs = "%s AND (%s)"%(cf, " OR ".join(map(tmpl.__mod__, evt)))
    return fs

In [80]:
pstatus = {player.name: collections.defaultdict(lambda: 0) for player in f.friendlies.players().friendlies}
pdamage = {player.name: 0 for player in f.friendlies.players().friendlies}

In [168]:
def actor(f, id):
    return f.actor(id).name

def fear_damage(f, cutoff, fs=crushing_fear()):
    pstatus = {player.name: 0 for player in f.friendlies.players().friendlies}
    admg = {player.name: 0 for player in f.friendlies.players().friendlies}
    ntick = {player.name: 0 for player in f.friendlies.players().friendlies}
    fdmg = {player.name: 0 for player in f.friendlies.players().friendlies}
    _actor = actor
    for e in f.events(fs, cutoff=cutoff):
        name = _actor(f, e.targetID)
        if e.type == "damage":
            stacks = pstatus[name]
            if stacks > 1:
                ticks = ntick[name]
                if not ticks:
                    ad = 400000  # base damage
                else:
                    ad = admg[name] / ticks
                fdmg[name] += e.amount + e.absorbed - ad
            else:
                ntick[name] += 1
                admg[name] += e.amount + e.absorbed
        elif e.type == "applydebuff":
            pstatus[name] += 1
        elif e.type == "applydebuffstack":
            pstatus[name] = int(e.stack)
        elif e.type == "removedebuff":
            pstatus[name] = 0
        else:
            raise TypeError(e.type)
    return fdmg

In [169]:
f = fights[4]
_=fear_damage(f, 4)
#f.events(crushing_fear())

In [170]:
pdamage = {}
pulls = {}
api.verbose = True

for f in fights:
    fl = f.friendlies.players().friendlies
    for pl in fl:
        if pl.name not in pdamage:
            pdamage[pl.name] = 0
            pulls[pl.name] = 0
            
fs = event_damage("Death Fog", "Unleashed Rage", "Edge of Annihiliation")
for cut, f in zip(cutoffs, fights):
    for p in f.friendlies.players().friendlies:
        pulls[p.name] += 1
    n, c = cut
    assert n == f.id
    events = f.events(fs, cutoff=c)
    for e in events:
        damage = e.amount + e.absorbed
        player = f.actor(e.targetID).name
        pdamage[player] += damage
    fdmg = fear_damage(f, c)
    for pname in fdmg:
        pdamage[pname] += fdmg[pname]


Requesting '/report/events/nC1YMGzb9kQwdDV6?start=315225&end=606076&filter=%28type%3D%22damage%22+AND+%28ability.name%3D%22Death+Fog%22+OR+ability.name%3D%22Unleashed+Rage%22+OR+ability.name%3D%22Edge+of+Annihiliation%22%29%29&cutoff=10&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/nC1YMGzb9kQwdDV6?start=315225&end=606076&filter=ability.name%3D%22Crushing+Fear%22+AND+%28%28type%3D%22damage%22%29+OR+%28type%3D%22applydebuff%22%29+OR+%28type%3D%22applydebuffstack%22%29+OR+%28type%3D%22removedebuff%22%29+OR+%28type%3D%22removedebuffstack%22%29%29&cutoff=10&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/nC1YMGzb9kQwdDV6?start=866129&end=1349451&filter=%28type%3D%22damage%22+AND+%28ability.name%3D%22Death+Fog%22+OR+ability.name%3D%22Unleashed+Rage%22+OR+ability.name%3D%22Edge+of+Annihiliation%22%29%29&cutoff=18&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/nC1YMGzb9kQwdDV6?start=1081115&end=1349451&filter=%28type%3D%22damag

Requesting '/report/events/nC1YMGzb9kQwdDV6?start=5590338&end=5866947&filter=ability.name%3D%22Crushing+Fear%22+AND+%28%28type%3D%22damage%22%29+OR+%28type%3D%22applydebuff%22%29+OR+%28type%3D%22applydebuffstack%22%29+OR+%28type%3D%22removedebuff%22%29+OR+%28type%3D%22removedebuffstack%22%29%29&cutoff=10&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/nC1YMGzb9kQwdDV6?start=5994389&end=6086037&filter=%28type%3D%22damage%22+AND+%28ability.name%3D%22Death+Fog%22+OR+ability.name%3D%22Unleashed+Rage%22+OR+ability.name%3D%22Edge+of+Annihiliation%22%29%29&cutoff=2&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/nC1YMGzb9kQwdDV6?start=5994389&end=6086037&filter=ability.name%3D%22Crushing+Fear%22+AND+%28%28type%3D%22damage%22%29+OR+%28type%3D%22applydebuff%22%29+OR+%28type%3D%22applydebuffstack%22%29+OR+%28type%3D%22removedebuff%22%29+OR+%28type%3D%22removedebuffstack%22%29%29&cutoff=2&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/event

In [171]:
dmg_taken = list(pdamage.items())
dmg_taken.sort(key=lambda t: t[1], reverse=True)
fmt = "%-15s: %-8s  | %-8s" 
print(fmt%("Player", "Total", "Per Pull"))
for player, dmg in dmg_taken:
    dmg /= 1000000
    ppdmg = dmg / pulls[player]
    sdmg = "%.2f"%dmg
    ppsdmg = "%.2f"%ppdmg
    print(fmt %(player, sdmg, ppsdmg))

Player         : Total     | Per Pull
Lowenherz      : 354.37    | 14.77   
Rosael         : 284.75    | 11.86   
Kôrboov        : 210.81    | 8.78    
Phue           : 198.83    | 8.28    
Renewed        : 183.88    | 7.66    
Warbill        : 178.38    | 7.43    
Namiyo         : 174.05    | 7.25    
Midnaught      : 155.33    | 6.47    
Dookú          : 147.69    | 6.15    
Grasping       : 147.34    | 6.14    
Kandlicious    : 140.29    | 5.85    
Abearcat       : 138.25    | 5.76    
Goblinguide    : 125.96    | 5.25    
Shinitai       : 121.73    | 5.07    
Owl            : 111.24    | 6.54    
Hrothgarth     : 80.50     | 3.50    
Ws             : 80.23     | 3.34    
Ragemeter      : 78.74     | 9.84    
Refanone       : 74.34     | 3.10    
Jermaine       : 73.12     | 3.05    
Nvrwastaken    : 61.30     | 2.55    


In [176]:
f.friendlies.friendlies[0].is_player()

True

In [183]:
players = list(fights[4].friendlies.players().id2f)
ed = [e for e in fights[4].events('type="death"', cutoff=4) if e.targetID in players]
for e in ed:
    print(actor(fights[4], e.targetID))

Dookú
Dookú
Kôrboov
Abearcat


In [104]:
dmg_taken[0]

'Kôrboov'

In [95]:
cutoffs = [
    (1, 3),
    (2, 10),
    (3, 18),
    (4, 5),
    (5, 3),
    (6, 13),
    (7, 5),
    (8, 4),
    (9, 5),
    (10, 2),
    (11, 3),
    (12, 10),
    (13, 10),
    (14, 2),
    (15, 11),
    (16, 11),
    (17, 4),
    (18, 3),
    (19, 6),
    (20, 4),
    (21, 3),
    (22, 3),
    (23, 8),
    (24, 4),
]

In [81]:
import webbrowser
"https://www.warcraftlogs.com/reports/nC1YMGzb9kQwdDV6/#fight=%d&type=deaths"
turl = "https://www.warcraftlogs.com/reports/nC1YMGzb9kQwdDV6/#fight=%d&type=deaths"
for i in range(1,25):
    url = turl%i
    webbrowser.open_new_tab(url)

In [83]:
print("wiped = [")
for i in range(1,25):
    print("    (%d, 0),"%i)
print("]")
    

wiped = [
    (1, 0),
    (2, 0),
    (3, 0),
    (4, 0),
    (5, 0),
    (6, 0),
    (7, 0),
    (8, 0),
    (9, 0),
    (10, 0),
    (11, 0),
    (12, 0),
    (13, 0),
    (14, 0),
    (15, 0),
    (16, 0),
    (17, 0),
    (18, 0),
    (19, 0),
    (20, 0),
    (21, 0),
    (22, 0),
    (23, 0),
    (24, 0),
]
